In [ ]:
# Install required dependencies
!pip install gradio tenacity llama-index azure-core azure-search-documents azure-storage-blob openai python-dotenv torch pypdf python-magic-bin

# Now import the required libraries
import os
import gradio as gr
import time
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
from llama_index.core import SimpleDirectoryReader, Settings, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set environment variables securely
os.environ['AZURE_OPENAI_API_KEY'] = os.getenv('AZURE_OPENAI_API_KEY')
os.environ['AZURE_OPENAI_ENDPOINT'] = os.getenv('AZURE_OPENAI_ENDPOINT')
os.environ['AZURE_SEARCH_API_KEY'] = os.getenv('AZURE_SEARCH_API_KEY')
os.environ['AZURE_SEARCH_ENDPOINT'] = os.getenv('AZURE_SEARCH_ENDPOINT')
os.environ['AZURE_STORAGE_CONNECTION_STRING'] = os.getenv('AZURE_STORAGE_CONNECTION_STRING')



@retry(
    wait=wait_exponential(multiplier=1, min=4, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(Exception)
)
def chat_with_retry(self, messages, **kwargs):
    try:
        return self._chat(messages, **kwargs)
    except Exception as e:
        print(f"An error occurred: {str(e)}. Retrying...")
        raise

# Replace the original chat method with the retry version
AzureOpenAI.chat = chat_with_retry

def get_router_query_engine(file_paths: list, model_deployment: str):
    llm = AzureOpenAI(
        model=model_deployment,
        deployment_name=model_deployment,
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )
    Settings.llm = llm
    Settings.embed_model = AzureOpenAIEmbedding(
        model="text-embedding-ada-002",
        deployment_name="text-embedding-ada-002",
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    )

    splitter = SentenceSplitter(chunk_size=1024)

    query_engine_tools = []
    for i, file_path in enumerate(file_paths, 1):
        documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
        nodes = splitter.get_nodes_from_documents(documents)

        summary_index = SummaryIndex(nodes)
        vector_index = VectorStoreIndex(nodes)

        summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize", use_async=True)
        vector_query_engine = vector_index.as_query_engine()

        summary_tool = QueryEngineTool.from_defaults(
            query_engine=summary_query_engine,
            description=f"Useful for summarization questions related to document {i}"
        )
        vector_tool = QueryEngineTool.from_defaults(
            query_engine=vector_query_engine,
            description=f"Useful for retrieving specific context from document {i}"
        )

        query_engine_tools.extend([summary_tool, vector_tool])

    router_query_engine = RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=query_engine_tools,
        verbose=True
    )

    return router_query_engine

def process_query(files, model_deployment, query):
    if not files:
        return "Please upload at least one PDF file."

    file_paths = [file.name for file in files]
    query_engine = get_router_query_engine(file_paths, model_deployment)

    try:
        response = query_engine.query(query)
        return str(response)
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio interface
iface = gr.Interface(
    fn=process_query,
    inputs=[
        gr.File(label="Upload PDFs", file_count="multiple"),
        gr.Dropdown(
            choices=[
                "gpt-35-turbo",
                "gpt-35-turbo-16k",
                "gpt-4",
                "gpt-4-32k"
            ],
            label="Choose Azure OpenAI Model Deployment"
        ),
        gr.Textbox(label="Enter your query")
    ],
    outputs=gr.Textbox(label="Response"),
    title="Agentic RAG with Azure Services",
    description="Upload multiple PDFs, select an Azure OpenAI model deployment, and ask questions about specific documents or all documents also can summarize the documents."
)

if __name__ == "__main__":
    iface.launch(share=True)